In [1]:
# CS3205 Networks - Assignment 3
# CS22B007, CS22B008
# TA group 1

import numpy as np
import heapq
from collections import deque, defaultdict


In [2]:

class Router:
    def __init__(self, router_id, coords):
        # Initialize a router with its ID, coordinates, neighbors (empty dict), and LSDB (empty dict)
        self.id = router_id
        self.coords = coords
        self.neighbors = {}  # {neighbor_id: distance}
        self.lsdb = {}  # {source_id: (sequence, links)}
        self.sequence = 0  # Sequence number for LSA generation

    def add_neighbor(self, neighbor_id, distance):
        # Add a neighbor to the router's neighbor list with the given distance
        self.neighbors[neighbor_id] = distance

    def generate_lsa(self):
        # Generate a Link State Advertisement (LSA) with the current sequence number and neighbor list
        self.sequence += 1
        return {
            'source': self.id,
            'seq': self.sequence,
            'links': self.neighbors.copy()
        }

    def process_lsa(self, lsa):
        # Process an incoming LSA: update LSDB if the sequence number is newer
        current_seq = self.lsdb.get(lsa['source'], (-1, {}))[0]
        if lsa['seq'] > current_seq:
            self.lsdb[lsa['source']] = (lsa['seq'], lsa['links'])
            return True  # Indicates the LSA was processed and forwarded
        return False  # Indicates the LSA was ignored (older sequence number)



In [3]:
from math import radians, cos, sin, asin, sqrt

def haversine(coord1, coord2, unit='km'):
    ### Computing haversine distances (spherical distances) ###
    
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Earth radius in km
    
    distance = c * r
    return distance * 1000 if unit == 'm' else distance

In [4]:
class Network:
    def __init__(self, vertices, T=0.3):
        # Initialize the network with routers and edges, randomly dropping T fraction of edges
        self.routers = {}
        self._build_network(vertices, T)

    def _build_network(self, vertices, T):
        # Create routers from the given vertices (IP to coordinates mapping)
        for idx, (ip, data) in enumerate(vertices.items(), 1):
            self.routers[idx] = Router(idx, data)

        # Create a complete graph and randomly drop edges based on threshold T
        router_ids = list(self.routers.keys())
        coords = [r.coords for r in self.routers.values()]

        # Calculate distance matrix using haversine distance
        dist_matrix = np.zeros((len(router_ids), len(router_ids)))
        for i in range(len(router_ids)):
            for j in range(i+1, len(router_ids)):
                # dist = geodesic(coords[i], coords[j]).km
                dist = haversine(coords[i][0], coords[j][0])
                if np.random.rand() > T:  # Keep edge with probability (1 - T)
                    dist_matrix[i][j] = dist
                    dist_matrix[j][i] = dist

        # Add neighbors based on the distance matrix
        for i, r1 in enumerate(router_ids):
            for j, r2 in enumerate(router_ids):
                if dist_matrix[i][j] > 0 and r1 != r2:
                    self.routers[r1].add_neighbor(r2, dist_matrix[i][j])
                    self.routers[r2].add_neighbor(r1, dist_matrix[i][j])

    def flood_lsas(self):
        # Simulate LSA flooding and count total messages and rounds for convergence
        message_count = 0
        rounds = 0
        queue = deque()

        # Generate initial LSAs for all routers and enqueue them for flooding
        for router in self.routers.values():
            lsa = router.generate_lsa()
            for neighbor in router.neighbors:
                queue.append((router.id, lsa, neighbor))
                message_count += 1

        # Process flooding in rounds until no more LSAs need to be forwarded
        while queue:
            rounds += 1
            current_round = list(queue)
            queue.clear()

            for sender, lsa, receiver in current_round:
                receiver_router = self.routers[receiver]
                if receiver_router.process_lsa(lsa):
                    # Forward LSA to all neighbors except the sender
                    for neighbor in receiver_router.neighbors:
                        if neighbor != sender:
                            queue.append((receiver, lsa, neighbor))
                            message_count += 1

            if not queue:
                break  # No more LSAs to forward

        return message_count, rounds

    def dijkstra(self, source_id):
        # Compute shortest paths from source_id using Dijkstra's algorithm
        distances = {rid: float('inf') for rid in self.routers}
        predecessors = {rid: None for rid in self.routers}
        distances[source_id] = 0

        heap = [(0, source_id)]

        while heap:
            current_dist, rid = heapq.heappop(heap)
            if current_dist > distances[rid]:
                continue

            router = self.routers[rid]
            for neighbor, cost in router.neighbors.items():
                new_dist = current_dist + cost
                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    predecessors[neighbor] = rid
                    heapq.heappush(heap, (new_dist, neighbor))

        return distances, predecessors

    def get_forwarding_table(self, source_id):
        # Generate a forwarding table for the given source router using Dijkstra's algorithm
        distances, predecessors = self.dijkstra(source_id)
        table = {}

        for dest_id in self.routers:
            if dest_id == source_id:
                continue

            path = []
            current = dest_id
            while predecessors[current] is not None:
                path.append(current)
                current = predecessors[current]
                if current == source_id:
                    break

            table[dest_id] = {
                'next_hop': path[-1] if path else None,
                'cost': distances[dest_id]
            }

        return table

    def get_routing_path(self, source_id, dest_id):
        # Get the routing path from source_id to dest_id using Dijkstra's algorithm
        _, predecessors = self.dijkstra(source_id)
        path = []
        current = dest_id

        while current is not None and current != source_id:
            path.append(current)
            current = predecessors.get(current)

        if current != source_id:
            return None  # No path exists

        path.append(source_id)
        return list(reversed(path))

In [5]:
import re
import statistics

def parse_traceroute(file_data):
    """Parse traceroute log data to generate IP mappings and path RTT tracking."""
    ip_to_info = {}  # Mapping of IP to [RTT list, city, country, ISP, (latitude, longitude)]
    
    # Static information for the IITM node
    iitm_coords = (12.99151, 80.23362)
    iitm_isp = "AS9498 BHARTI Airtel Ltd."


    # Regex patterns
    ip_regex = r"(\d+\.\d+\.\d+\.\d+)"
    location_regex = r"\(([^()]*?), ([^()]*?), (AS\d+ [^()]*?), loc: ([-\d\.]+),([-\d\.]+)\)"
    local_regex = r"\(Local / ([^()]*?)\)"
    rtt_regex = r"(\d+\.\d+) ms"

    # Split the data into different iterations
    iterations = file_data.split("Iteration ")
    iterations = [iteration.strip() for iteration in iterations if iteration.strip()]

    # Iterate through each traceroute iteration
    for iteration in iterations:
        lines = iteration.split("\n")
        current_path = []
        current_rtts = []

        for line in lines:
            if "traceroute to" in line or "------" in line or not line.strip():
                continue

            # Extract hop number
            hop_match = re.match(r"^\s*(\d+)", line)
            if not hop_match:
                continue

            # Find IPs in the current line
            ips = re.findall(ip_regex, line)
            if not ips:
                continue

            # Track the first IP of the hop (or None if *)
            hop_ip = ips[0] if "*" not in line.split()[1] else None
            current_path.append(hop_ip)

            # Capture RTTs for the current hop
            rtts = re.findall(rtt_regex, line)
            if rtts:
                avg_rtt = statistics.mean([float(rtt) for rtt in rtts])
                current_rtts.append(avg_rtt)
            else:
                current_rtts.append(None)

            # Process each IP in the line
            for ip in ips:
                if ip not in ip_to_info:
                    # Try extracting location info for the IP
                    location_match = re.search(f"{ip} {location_regex}", line)
                    local_match = re.search(f"{ip} {local_regex}", line)

                    if location_match:
                        _, _, _, lat, lon = location_match.groups()
                        ip_to_info[ip] = [(float(lat), float(lon))]
                    elif local_match:
                        ip_to_info[ip] = [ iitm_coords]


    return ip_to_info

In [6]:
############# IMPLEMENTATION DETAILS #########################################################################################################################
########### Step 1: Network Initialization
##### We construct the physical network topology by:
## Router.__init__() creates router instances with unique IDs and geographic coordinates, initializing empty neighbor lists and Link State Databases (LSDB)
## Network._build_network() generates a complete graph where each router initially connects to all others, using:
# Haversine formula for accurate geospatial distance calculations
# Probabilistic edge removal (threshold T) to create realistic sparse connectivity
## add_neighbor() establishes symmetric, weighted connections in the router's neighbor table, ensuring bidirectional reachability

########### Step 2: LSA Flooding Protocol
##### The link-state advertisement process implements:
## generate_lsa() produces timestamped advertisements containing:
# Current neighbor topology
# Strictly increasing sequence numbers for version control

## process_lsa() enforces consistency by:
# Validating sequence numbers against the LSDB
# Implementing a store-and-forward policy for new information

## flood_lsas() manages the controlled flood propagation:
# Uses BFS-like rounds with message queues
# Implements split horizon (never rebroadcast to sender)
# Tracks convergence via message counts and rounds

########### Step 3: Path Computation Engine
##### After topology convergence, we:
## dijkstra() computes shortest paths using:
# Priority queue (min-heap) for O(E + VlogV) efficiency
# Maintains both distance and predecessor maps

## get_forwarding_table() generates practical routing tables by:
# Extracting next-hop routers from predecessor chains
# Including both next-hop and total path cost

## get_routing_path() provides end-to-end path visibility by:
# Backtracing from destination to source
# Reversing the path for intuitive presentation
# Handling disconnected nodes gracefully
##############################################################################################################################################################

In [7]:
# Read the traceroute log and parse the data
with open('traceroute_log.txt', 'r') as file:
    content = file.read()

vertices = parse_traceroute(content)

# print(vertices)

In [8]:

# Initialize network with 30% edge removal
network = Network(vertices, T=0.3)

# Simulate LSA flooding
total_msgs, rounds = network.flood_lsas()
print(f"Total LSA Messages: {total_msgs}")
print(f"Convergence Rounds: {rounds}")
print(f"LSDB Size per Router: {len(network.routers)} entries") # Number of entries in the LSDB

# Example usage
router_id = 3
print(f"\nForwarding Table for Router {router_id}:")
table = network.get_forwarding_table(router_id)
print(f"{'Destination':<12}{'Next Hop':<10}{'Cost':<10}")
for dest, info in table.items():
    print(f"{dest:<12}{info['next_hop'] or '-':<10}{info['cost'] or '∞':<10.2f}")

source = 1
dest = 3
path = network.get_routing_path(source, dest)
if path:
    print(f"\nRouting Path from {source} to {dest}: {' → '.join(map(str, path))}")
else:
    print(f"\nNo path exists from {source} to {dest}")


Total LSA Messages: 6302244
Convergence Rounds: 3
LSDB Size per Router: 213 entries

Forwarding Table for Router 3:
Destination Next Hop  Cost      
1           12        2072.34   
2           22        2072.34   
4           12        2072.34   
5           12        2072.34   
6           12        2072.34   
7           22        2080.67   
8           12        2072.34   
9           12        2072.34   
10          12        3108.51   
11          11        2907.96   
12          12        1036.17   
13          13        2907.96   
14          143       7786.30   
15          12        4945.88   
16          16        7785.91   
17          12        2072.34   
18          12        2072.34   
19          12        2072.34   
20          22        4945.88   
21          21        2907.96   
22          22        1036.17   
23          60        2072.34   
24          22        2068.91   
25          12        2072.34   
26          12        4945.88   
27          60        4945